In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
items           = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')item_cnt_month = transactions.groupby(['shop_id', 'item_id', 'month', 'year'])['item_cnt_day'].sum().reset_index(name ='item_cnt_month')
item_cnt_month.head()
item_cnt_month.shape
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train_v2.csv')
test            = pd.read_csv('test.csv')
sample_submission            = pd.read_csv('sample_submission.csv')

In [4]:
transactions = sales_train.copy()
transactions[['day','month', 'year']] = transactions['date'].str.split('.', expand=True).astype(int)

item_cnt_month = transactions.groupby(['shop_id', 'item_id', 'month', 'year'])['item_cnt_day'].sum().reset_index(name ='item_cnt_month')

In [5]:
month_year = set(list(zip(item_cnt_month.month, item_cnt_month.year)))

years = list(set([y for (m,y) in month_year]))
years.sort()
months = list(range(1,13))

date_to_int = {}

i = 1
for year in years:
    for month in months:
        date_to_int[(month,year)] = i
        i+=1
        
def get_date_int(row):
    return date_to_int[(row['month'], row['year'])]

item_cnt_month['date_counter'] = item_cnt_month.apply(get_date_int, axis=1)

In [15]:
w_cat_ids = item_cnt_month.set_index('item_id').join(items.set_index('item_id'))
w_cat_ids['shop_cat_tuple'] = w_cat_ids.apply(lambda row: (row.shop_id, row.item_category_id),axis=1)

In [7]:
w_cat_ids.sample(10)

,shop_id,month,year,item_cnt_month,date_counter,item_name,item_category_id,shop_cat_tuple
item_id,,,,,,,,
10469,44,3,2014,1.0,15,ГОРДОСТЬ И ПРЕДУБЕЖДЕНИЕ (Кармен),40,"(44, 40)"
10058,31,6,2013,1.0,6,ВРАТА ДРАКОНА (3D BD),38,"(31, 38)"
4178,37,3,2014,2.0,15,Kaspersky Internet Security Multi-Device Russi...,75,"(37, 75)"
10928,15,12,2013,1.0,12,ДЕТЯМ Песенки для самых маленьких 2,55,"(15, 55)"
19089,19,8,2014,1.0,20,СТАРТРЕК ВОЗМЕЗДИЕ (BD),37,"(19, 37)"
1171,55,2,2014,1.0,14,ABBYY Lingvo x5 9 языков Домашняя версия (ABBY...,76,"(55, 76)"
9589,27,8,2013,1.0,8,ВВЕРХ (регион),40,"(27, 40)"
11194,57,3,2013,23.0,3,ДОСПЕХИ БОГА 3: МИССИЯ ЗОДИАК,40,"(57, 40)"
16234,34,11,2014,1.0,23,Настольная игра Хоббит Карточная игра арт. 1047,65,"(34, 65)"


In [19]:
w_cat_ids['item_mean_month'] = w_cat_ids.groupby(w_cat_ids.index)['item_cnt_month'].mean()
w_cat_ids['shop_mean_month'] = w_cat_ids.groupby(w_cat_ids.shop_id)['item_cnt_month'].transform(np.mean)
w_cat_ids['category_mean_month'] = w_cat_ids.groupby(w_cat_ids.item_category_id)['item_cnt_month'].transform(np.mean)


In [26]:
november = w_cat_ids[w_cat_ids.month == 11]

item_november_means = november.groupby(november.index)['item_cnt_month'].mean()
item_november_means = item_id_november_means.reindex(index=w_cat_ids.index)
w_cat_ids['item_mean_november'] = item_november_means

category_november_means = november.groupby(november.item_category_id)['item_cnt_month'].mean()
category_november_means = category_november_means.reindex(index=w_cat_ids.index)
w_cat_ids['category_mean_november'] = category_november_means


shop_november_means = november.groupby(november.shop_id)['item_cnt_month'].mean()
shop_november_means = shop_november_means.reindex(index=w_cat_ids.index)
w_cat_ids['shop_mean_november'] = shop_november_means



In [23]:
w_cat_ids['item_shop_mean'] = w_cat_ids.groupby([w_cat_ids.index, w_cat_ids.shop_id])['item_cnt_month'].transform(np.mean)
w_cat_ids['category_shop_mean'] = w_cat_ids.groupby([w_cat_ids.item_category_id, w_cat_ids.shop_id])['item_cnt_month'].transform(np.mean)


In [34]:
Q3 = w_cat_ids[w_cat_ids.month.isin([10,11,12])]

item_id_Q3_means = Q3.groupby(Q3.index)['item_cnt_month'].mean()
item_id_Q3_means = item_id_Q3_means.reindex(index=w_cat_ids.index)
w_cat_ids['item_mean_Q3'] = item_id_Q3_means

category_Q3_means = Q3.groupby(Q3.item_category_id)['item_cnt_month'].mean()
category_Q3_means = category_Q3_means.reindex(index=w_cat_ids.index)
w_cat_ids['category_mean_Q3'] = category_Q3_means


shop_Q3_means = Q3.groupby(Q3.shop_id)['item_cnt_month'].mean()
shop_Q3_means = shop_Q3_means.reindex(index=w_cat_ids.index)
w_cat_ids['shop_mean_Q3'] = shop_Q3_means



In [45]:
w_cat_ids[w_cat_ids.category_mean_Q3.notna()]

,shop_id,month,year,item_cnt_month,date_counter,item_name,item_category_id,shop_cat_tuple,item_mean_november,category_mean_november,shop_mean_november,item_mean_month,shop_mean_month,category_mean_month,item_shop_mean,category_shop_mean,item_mean_Q3,category_mean_Q3,shop_Q3_november,shop_mean_Q3
item_id,,,,,,,,,,,,,,,,,,,,
2,54,8,2014,1.0,20,***В ЛУЧАХ СЛАВЫ (UNV) D,40,"(54, 40)",1.0,1.732919,2.567500,1.000000,2.636404,1.886919,1.000000,2.536949,1.000000,2.127439,2.347037,2.347037
2,54,11,2014,1.0,23,***В ЛУЧАХ СЛАВЫ (UNV) D,40,"(54, 40)",1.0,1.732919,2.567500,1.000000,2.636404,1.886919,1.000000,2.536949,1.000000,2.127439,2.347037,2.347037
3,54,7,2014,1.0,19,***ГОЛУБАЯ ВОЛНА (Univ) D,40,"(54, 40)",NaN,3.087459,1.903558,1.000000,2.636404,1.886919,1.000000,2.536949,NaN,3.895792,1.815280,1.815280
3,54,8,2014,1.0,20,***ГОЛУБАЯ ВОЛНА (Univ) D,40,"(54, 40)",NaN,3.087459,1.903558,1.000000,2.636404,1.886919,1.000000,2.536949,NaN,3.895792,1.815280,1.815280
4,54,9,2014,1.0,21,***КОРОБКА (СТЕКЛО) D,40,"(54, 40)",NaN,1.877551,2.117865,1.000000,2.636404,1.886919,1.000000,2.536949,NaN,1.683168,2.063636,2.063636
5,54,12,2014,1.0,24,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,40,"(54, 40)",NaN,1.352518,1.876241,1.000000,2.636404,1.886919,1.000000,2.536949,1.000000,1.590406,1.952694,1.952694
6,54,7,2014,1.0,19,***УДАР ПО ВОРОТАМ (UNI) D,40,"(54, 40)",NaN,2.254386,2.485273,1.000000,2.636404,1.886919,1.000000,2.536949,NaN,2.684359,2.379081,2.379081
7,54,12,2014,1.0,24,***УДАР ПО ВОРОТАМ-2 (UNI) D,40,"(54, 40)",NaN,1.461538,2.297433,1.000000,2.636404,1.886919,1.000000,2.536949,1.000000,2.698644,2.369630,2.369630
8,54,8,2014,1.0,20,***ЧАЙ С МУССОЛИНИ D,40,"(54, 40)",NaN,NaN,NaN,1.000000,2.636404,1.886919,1.000000,2.536949,NaN,28.363636,NaN,NaN


In [51]:
w_cat_ids.sample(10)

,shop_id,month,year,item_cnt_month,date_counter,item_name,item_category_id,shop_cat_tuple,item_mean_november,category_mean_november,shop_mean_november,item_mean_month,shop_mean_month,category_mean_month,item_shop_mean,category_shop_mean,item_mean_Q3,category_mean_Q3,shop_Q3_november,shop_mean_Q3
item_id,,,,,,,,,,,,,,,,,,,,
1584,57,10,2015,3.0,34,Assassin's Creed: Синдикат. Специальное издани...,24,"(57, 24)",NaN,NaN,NaN,2.440000,2.352881,1.807923,3.000000,1.893333,2.440000,NaN,NaN,NaN
8918,15,8,2014,1.0,20,БЛОНДИНКА В ЭФИРЕ,40,"(15, 40)",1.000000,NaN,NaN,1.654867,2.240152,1.886919,1.000000,1.700594,1.130435,NaN,NaN,NaN
5022,54,4,2013,1.0,4,NETREBKO ANNA Souvenirs,55,"(54, 55)",1.250000,NaN,NaN,1.087719,2.636404,1.404199,1.333333,1.697668,1.083333,NaN,NaN,NaN
19068,42,7,2014,1.0,19,СТАЛИНГРАД (регион),40,"(42, 40)",1.500000,NaN,NaN,1.468927,2.548918,1.886919,1.222222,1.972694,1.344828,NaN,NaN,NaN
6247,13,6,2014,1.0,18,SCORPIONS MTV Unplugged In Athens 2CD,55,"(13, 55)",1.294118,NaN,NaN,1.750623,1.632631,1.404199,2.000000,1.317013,1.740741,NaN,NaN,NaN
19185,59,10,2013,1.0,10,СУМЕРКИ (BD),37,"(59, 37)",1.000000,NaN,NaN,1.050633,1.952846,1.510428,1.000000,1.206836,1.066667,NaN,NaN,NaN
14429,19,8,2013,8.0,8,МАША И МЕДВЕДЬ. ВИТАМИН РОСТА,40,"(19, 40)",1.388889,NaN,NaN,4.322314,2.005324,1.886919,5.833333,1.591390,1.796296,NaN,NaN,NaN
5643,15,2,2015,1.0,26,PS3: Контроллер игровой беспроводной черный (D...,2,"(15, 2)",2.661765,NaN,NaN,2.724388,2.240152,1.846335,2.344828,1.630841,3.484979,NaN,NaN,NaN
12965,16,7,2013,1.0,7,КРУГ ИРИНА & КОРОЛЕВ ВИКТОР Городские встечи,55,"(16, 55)",1.000000,NaN,NaN,1.103448,1.968917,1.404199,1.000000,1.275735,1.125000,NaN,NaN,NaN
